In [1]:
!pip -q install pyvista trimesh scikit-learn scipy scikit-image numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.6/145.6 MB 6.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pyvista as pv
import trimesh

from scipy.spatial import cKDTree
from scipy.ndimage import gaussian_filter1d

from sklearn.decomposition import PCA
from skimage.measure import EllipseModel, ransac


In [46]:
VTK_PATH = "prestress2800000.vtk"
STL_PATH = "drillhead2800000.stl"

OUT_STL  = "borehole_centerline_ransac.stl"

# If Zamir's symmetry trick is applicable, duplicate points mirrored about x-z plane (y -> -y)
USE_MIRROR_XZ = True

# Point selection near tool (units must match your data)
# If pipe dia is ~25mm and aligned, start with 6-15mm.
DIST_THRESH = 0.12

# Stationing along tunnel (controls how many slices)
# Smaller -> more slices, slower. Start around 2.5 to 5.0 (mm-ish).
STEP_ALONG = 0.01

# Perpendicular slab thickness (how far from slice plane you accept points)
SLAB_HALF_THICK = 0.015  # accept points with |distance to plane| < this

# Minimum points needed in a slice to attempt a fit
MIN_SLICE_PTS = 30

# RANSAC params for ellipse
RANSAC_MIN_SAMPLES = 8
RANSAC_RESID_TOL   = 0.01
RANSAC_MAX_TRIALS  = 120

# Reject slices with too few inliers
MIN_INLIER_FRAC = 0.15

# Ring resolution for lofting
RING_N = 40

# Optional guardrails for ellipse size (helps reject crazy fits)
# Set to None if you don't want bounds.
MIN_RADIUS = 2.0
MAX_RADIUS = 40.0

In [66]:
def mirror_about_xz(points: np.ndarray) -> np.ndarray:
    p = points.copy()
    p[:, 1] *= -1.0
    return p

def normalize(v: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(v)
    if n < eps:
        return v * 0.0
    return v / n

def get_plane_basis(n: np.ndarray):
    """Given normal vector n, return orthonormal basis (e1,e2) spanning plane."""
    n = normalize(n)
    v = np.array([1.0, 0.0, 0.0])
    if abs(np.dot(v, n)) > 0.9:
        v = np.array([0.0, 1.0, 0.0])
    e1 = normalize(np.cross(n, v))
    e2 = normalize(np.cross(n, e1))
    return e1, e2

def ellipse_ring(xc, yc, a, b, theta, n=64):
    """Return (n,2) points on ellipse in 2D."""
    t = np.linspace(0, 2*np.pi, n, endpoint=False)
    x = a*np.cos(t)
    y = b*np.sin(t)
    ct, st = np.cos(theta), np.sin(theta)
    xr = ct*x - st*y + xc
    yr = st*x + ct*y + yc
    return np.column_stack([xr, yr])

def loft_rings_to_mesh(rings3d):
    import trimesh
    import numpy as np

    RING_N = rings3d[0].shape[0]

    verts = np.vstack(rings3d)
    faces = []

    def idx(i, j):
        return i*RING_N + (j % RING_N)

    for i in range(len(rings3d)-1):
        for j in range(RING_N):
            a0 = idx(i, j)
            a1 = idx(i, j+1)
            b0 = idx(i+1, j)
            b1 = idx(i+1, j+1)

            faces.append([a0, b0, a1])
            faces.append([a1, b0, b1])

    faces = np.array(faces, dtype=np.int64)

    return trimesh.Trimesh(verts, faces, process=False)

def fit_circle_3pts(p1, p2, p3, eps=1e-12):
    (x1,y1),(x2,y2),(x3,y3) = p1, p2, p3

    A = np.array([
        [x2-x1, y2-y1],
        [x3-x1, y3-y1]
    ], dtype=float)

    B = np.array([
        0.5*((x2**2 - x1**2) + (y2**2 - y1**2)),
        0.5*((x3**2 - x1**2) + (y3**2 - y1**2))
    ], dtype=float)

    if abs(np.linalg.det(A)) < eps:
        return None

    cx, cy = np.linalg.solve(A, B)
    r = np.sqrt((cx-x1)**2 + (cy-y1)**2)

    if not np.isfinite(r) or r <= 0:
        return None

    return cx, cy, r


def ransac_circle(pts2d, n_trials=150, tol=0.01, min_inlier_frac=0.12, rng=None):

    if rng is None:
        rng = np.random.default_rng(0)

    N = pts2d.shape[0]
    if N < 20:
        return None

    best = None
    best_inliers = 0
    best_mask = None

    for _ in range(n_trials):

        idx = rng.choice(N, size=3, replace=False)

        model = fit_circle_3pts(
            pts2d[idx[0]],
            pts2d[idx[1]],
            pts2d[idx[2]]
        )

        if model is None:
            continue

        cx, cy, r = model

        d = np.sqrt((pts2d[:,0]-cx)**2 + (pts2d[:,1]-cy)**2)
        resid = np.abs(d - r)

        mask = resid < tol
        cnt = mask.sum()

        if cnt > best_inliers:
            best_inliers = cnt
            best = (cx, cy, r)
            best_mask = mask

    if best is None:
        return None

    if best_inliers / N < min_inlier_frac:
        return None

    return (*best, best_mask)


In [67]:
soil = pv.read(VTK_PATH)
points = np.asarray(soil.points)

tool = trimesh.load_mesh(STL_PATH)
tool_pts = np.asarray(tool.vertices)

print("Soil points:", points.shape)
print("Tool vertices:", tool_pts.shape)

if USE_MIRROR_XZ:
    points = np.vstack([points, mirror_about_xz(points)])
    print("After mirroring:", points.shape)

Soil points: (696791, 3)
Tool vertices: (3658, 3)
After mirroring: (1393582, 3)


In [68]:
tree = cKDTree(tool_pts)
dists, _ = tree.query(points, k=1)

MAX_PTS = 250_000   # safe for laptop

if hole_pts.shape[0] > MAX_PTS:
    idx = np.random.choice(hole_pts.shape[0], MAX_PTS, replace=False)
    hole_pts = hole_pts[idx]

print("Hole candidate points:", hole_pts.shape)

if hole_pts.shape[0] < 1000:
    print("\nWARNING: Very few near-tool points. Try increasing DIST_THRESH.\n")

print("Distance stats:")
print(" min:", dists.min())
print(" median:", np.median(dists))
print(" mean:", dists.mean())
print(" max:", dists.max())

Hole candidate points: (250000, 3)
Distance stats:
 min: 0.0003335340419873248
 median: 0.08366718310710813
 mean: 0.17280180280634566
 max: 0.6136396909241136


In [69]:
pca = PCA(3)
pca.fit(hole_pts)

main_dir = normalize(pca.components_[0])
center0 = hole_pts.mean(axis=0)

proj = (hole_pts - center0) @ main_dir

s_min, s_max = proj.min(), proj.max()
stations = np.arange(s_min, s_max, STEP_ALONG)

print("Stations:", len(stations), "range:", (s_min, s_max))

Stations: 132 range: (np.float32(-0.8247145), np.float32(0.49220088))


In [70]:
centers = []
station_vals = []

half = STEP_ALONG / 2.0

for s in stations:
    mask = np.abs(proj - s) < half
    chunk = hole_pts[mask]
    if chunk.shape[0] < MIN_SLICE_PTS:
        continue
    centers.append(chunk.mean(axis=0))
    station_vals.append(s)

centers = np.asarray(centers)
station_vals = np.asarray(station_vals)

print("Centerline raw points:", centers.shape)

# Smooth centerline
centers_smooth = gaussian_filter1d(centers, sigma=2.5, axis=0)

d = np.gradient(centers_smooth, axis=0)
tangents = d / np.linalg.norm(d, axis=1, keepdims=True)


Centerline raw points: (132, 3)


In [72]:
fits = []
rings3d = []
aspect_ratios = []

from scipy.spatial import cKDTree
hole_tree = cKDTree(hole_pts)

rng = np.random.default_rng(0)

# Tuned for your scale
BALL_R = 0.06
SLAB_HALF_THICK = 0.02
MIN_SLICE_PTS = 40
RANSAC_TRIALS = 160
CIRCLE_TOL = 0.008
MIN_INLIER_FRAC = 0.12
RING_N = 48


for i in range(len(centers_smooth)):

    c = centers_smooth[i]
    n = tangents[i]

    e1, e2 = get_plane_basis(n)

    # Get local neighborhood
    idxs = hole_tree.query_ball_point(c, r=BALL_R)

    if len(idxs) < MIN_SLICE_PTS:
        continue

    local = hole_pts[idxs]

    # Slab filter
    rel = local - c
    dist = np.abs(rel @ n)

    local = local[dist < SLAB_HALF_THICK]

    if local.shape[0] < MIN_SLICE_PTS:
        continue

    # Project to 2D
    rel2 = local - c
    x = rel2 @ e1
    y = rel2 @ e2

    pts2d = np.column_stack([x,y])

    # Subsample
    if pts2d.shape[0] > 500:
        sel = rng.choice(pts2d.shape[0], 500, replace=False)
        pts2d = pts2d[sel]

    # RANSAC circle
    out = ransac_circle(
        pts2d,
        n_trials=RANSAC_TRIALS,
        tol=CIRCLE_TOL,
        min_inlier_frac=MIN_INLIER_FRAC,
        rng=rng
    )

    if out is None:
        continue

    cx, cy, r, inliers = out

    # Aspect proxy (optional)
    in_pts = pts2d[inliers]

    if in_pts.shape[0] >= 20:
        cov = np.cov(in_pts.T)
        eig = np.linalg.eigvalsh(cov)
        eig = np.sort(eig)[::-1]
        aspect = np.sqrt(eig[0]/max(eig[1],1e-12))
    else:
        aspect = 1.0

    # Build ring
    t = np.linspace(0, 2*np.pi, RING_N, endpoint=False)

    ring2d = np.column_stack([
        cx + r*np.cos(t),
        cy + r*np.sin(t)
    ])

    ring3d = (
        c
        + ring2d[:,0,None]*e1
        + ring2d[:,1,None]*e2
    )

    fits.append({"i": i, "r": r})
    rings3d.append(ring3d)
    aspect_ratios.append(aspect)


print("Successful RANSAC circle slice fits:", len(fits))

Successful RANSAC circle slice fits: 132


In [73]:
radii = np.array([f["r"] for f in fits])

r_smooth = savgol_filter(radii, 11, 2)

centers_fit = np.array([
    centers_smooth[f["i"]] for f in fits
])

centers_fit_smooth = gaussian_filter1d(
    centers_fit,
    sigma=2.5,     # stronger smoothing
    axis=0
)


In [74]:
d = np.gradient(centers_fit_smooth, axis=0)

tangents_smooth = d / np.linalg.norm(d, axis=1, keepdims=True)

In [75]:
rings_smooth = []

for k, f in enumerate(fits):

    c = centers_fit_smooth[k]
    n = tangents_smooth[k]

    e1, e2 = get_plane_basis(n)

    r = r_smooth[k]

    t = np.linspace(0,2*np.pi,RING_N,endpoint=False)

    ring2d = np.column_stack([
        r*np.cos(t),
        r*np.sin(t)
    ])

    ring3d = c + ring2d[:,0,None]*e1 + ring2d[:,1,None]*e2

    rings_smooth.append(ring3d)

In [76]:
mesh = loft_rings_to_mesh(rings_smooth)

mesh.export("borehole_final_smooth.stl")

print("Saved borehole_final_smooth.stl")

Saved borehole_final_smooth.stl
